<a href="https://colab.research.google.com/github/Rickyimmanuel/MergedMistral/blob/main/Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate peft bitsandbytes
from huggingface_hub import login
login(token="hugging-face-access-token")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


HTTPError: Invalid user token.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)


In [ ]:
from peft import PeftModel
import torch

# Load summarization LoRA
summarizer_model_id = "waelChafei/resume-summarization-finetuned-mistral-7b"
model_summarizer = PeftModel.from_pretrained(base_model, summarizer_model_id)

# Loop for resume text input
while True:
    resume_text = input("\n📝 Paste resume text for summarization (or type 'exit'): ")
    if resume_text.lower() == "exit":
        print("Exiting summarization loop.")
        break

    # prompt structure
    wrapped_prompt = f"""### Instruction:
Summarize the following resume in 2 lines.

### Input:
{resume_text}

### Response:"""

    inputs = tokenizer(wrapped_prompt, return_tensors="pt").to(model_summarizer.device)

    with torch.no_grad():
        outputs = model_summarizer.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract clean response
    if "### Response:" in full_output:
        response = full_output.split("### Response:")[-1].strip()
    else:
        response = full_output.strip()

    print("\n📤 Summary:\n", response)

adapter_config.json:   0%|          | 0.00/751 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]


📝 Paste resume text for summarization (or type 'exit'): Summarize the resume in 2 lines: John Doe is a software engineer with over 5 years of experience at Google working on scalable ML systems. He graduated with a B.Sc in Computer Science from MIT in 2018. His key skills include Python, TensorFlow, distributed systems, and MLOps. He has contributed to several open-source projects and has strong expertise in backend engineering. You can contact him at john.doe@gmail.com.

📤 Summary:
 Software Engineer at Google with 5 years of experience in scalable ML systems. Holds a B.Sc in Computer Science from MIT (2018). Skilled in Python, TensorFlow, distributed systems, and MLOps. Contributed to open-source projects and proficient in backend engineering. Contact: john.doe@gmail.com.

📝 Paste resume text for summarization (or type 'exit'): exit
Exiting summarization loop.


In [ ]:
# Re-initialize base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load entity extraction LoRA
entity_model_id = "Pennlaine/Mistral-7B-v02-Entity-Extraction"
model_entity = PeftModel.from_pretrained(base_model, entity_model_id)

# Prompt-based test
while True:
    prompt = input("\n📝 Enter resume prompt for entity extraction model (or 'exit'): ")
    if prompt.lower() == "exit":
        break

    inputs = tokenizer(prompt, return_tensors="pt").to(model_entity.device)

    with torch.no_grad():
        outputs = model_entity.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n📤 Response:\n", response)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/42.0M [00:00<?, ?B/s]


📝 Enter resume prompt for entity extraction model (or 'exit'): Exract key details like -name -experience -education -email from the resume :John Doe is a software engineer with over 5 years of experience at Google working on scalable ML systems. He graduated with a B.Sc in Computer Science from MIT in 2018. His key skills include Python, TensorFlow, distributed systems, and MLOps. He has contributed to several open-source projects and has strong expertise in backend engineering. You can contact him at john.doe@gmail.com.

📤 Response:
 Exract key details like -name -experience -education -email from the resume :John Doe is a software engineer with over 5 years of experience at Google working on scalable ML systems. He graduated with a B.Sc in Computer Science from MIT in 2018. His key skills include Python, TensorFlow, distributed systems, and MLOps. He has contributed to several open-source projects and has strong expertise in backend engineering. You can contact him at john.doe@gmail

In [ ]:

!pip install -q transformers accelerate bitsandbytes


from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch


model_id = "Impulse007/merged-mistral-lora2"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer


def format_prompt(text, mode="both"):
    text = text.strip()
    if mode == "summarize":
        return f"Summarize the following resume. Only provide a short professional summary.\n\nResume:\n{text}"
    elif mode == "extract":
        return (
            "Extract structured information from the following resume. "
            "Format your response with key fields like:\n"
            "- Name\n- Role\n- Experience\n- Education\n- Skills\n- Contact\n\nResume:\n" + text
        )
    elif mode == "both":
        return (
            "Summarize and extract key information from the following resume. "
            "First, provide a short professional summary. Then list extracted fields clearly as:\n"
            "- Name\n- Role\n- Experience\n- Education\n- Skills\n- Contact\n\nResume:\n" + text
        )
    else:
        return f"Summarize and extract details:\n\n{text}"



def process_resumes_interactively():
    print("💡 Enter 'exit' anytime to stop.\n")

    while True:
        # Select mode
        mode = input("🔧 Enter mode (summarize / extract / both): ").strip().lower()
        if mode == "exit":
            break
        if mode not in ["summarize", "extract", "both"]:
            print("❗ Invalid mode. Choose from summarize / extract / both.")
            continue

        # Input resume text
        resume_text = input("📝 Paste the resume text: ").strip()
        if resume_text.lower() == "exit":
            break

        # Format and run
        prompt = format_prompt(resume_text, mode=mode)
        output = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)[0]["generated_text"]

        # Display result
        print("\n📤 Output:\n" + "-"*50)
        print(output)
        print("-"*50 + "\n")


process_resumes_interactively()


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


💡 Enter 'exit' anytime to stop.

🔧 Enter mode (summarize / extract / both): summarize
📝 Paste the resume text: John Doe is a software engineer with over 5 years of experience at Google working on scalable ML systems. He graduated with a B.Sc in Computer Science from MIT in 2018. His key skills include Python, TensorFlow, distributed systems, and MLOps. He has contributed to several open-source projects and has strong expertise in backend engineering. You can contact him at john.doe@gmail.com.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📤 Output:
--------------------------------------------------
Summarize the following resume. Only provide a short professional summary.

Resume:
John Doe is a software engineer with over 5 years of experience at Google working on scalable ML systems. He graduated with a B.Sc in Computer Science from MIT in 2018. His key skills include Python, TensorFlow, distributed systems, and MLOps. He has contributed to several open-source projects and has strong expertise in backend engineering. You can contact him at john.doe@gmail.com.

Professional Summary:
Experienced Software Engineer with a B.Sc in Computer Science from MIT and over 5 years of experience at Google in ML systems development. Skilled in Python, TensorFlow, distributed systems, and MLOps. Strong background in backend engineering with contributions to open-source projects.
--------------------------------------------------

🔧 Enter mode (summarize / extract / both): extract
📝 Paste the resume text: John Doe is a software engine

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📤 Output:
--------------------------------------------------
Extract structured information from the following resume. Format your response with key fields like:
- Name
- Role
- Experience
- Education
- Skills
- Contact

Resume:
John Doe is a software engineer with over 5 years of experience at Google working on scalable ML systems. He graduated with a B.Sc in Computer Science from MIT in 2018. His key skills include Python, TensorFlow, distributed systems, and MLOps. He has contributed to several open-source projects and has strong expertise in backend engineering. You can contact him at john.doe@gmail.com.

Name: John Doe
Role: Software Engineer
Experience:
- Software Engineer, Google (2018-present)

Education:
- B.Sc in Computer Science, Massachusetts Institute of Technology (MIT) (2014-2018)

Skills:
- Python
- TensorFlow
- Distributed systems
- MLOps

Contact: john.doe@gmail.com
--------------------------------------------------

🔧 Enter mode (summarize / extract / both): both
📝 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📤 Output:
--------------------------------------------------
Summarize and extract key information from the following resume. First, provide a short professional summary. Then list extracted fields clearly as:
- Name
- Role
- Experience
- Education
- Skills
- Contact

Resume:
John Doe is a software engineer with over 5 years of experience at Google working on scalable ML systems. He graduated with a B.Sc in Computer Science from MIT in 2018. His key skills include Python, TensorFlow, distributed systems, and MLOps. He has contributed to several open-source projects and has strong expertise in backend engineering. You can contact him at john.doe@gmail.com.

Professional Summary:
John Doe is a software engineer with 5 years of experience at Google, specialized in scalable machine learning systems.

- Name: John Doe
- Role: Software Engineer
- Experience:
  - Software Engineer, Google (2018-Present)
- Education:
  - B.Sc in Computer Science, MIT (2014-2018)
- Skills:
  - Python
  - Tenso